# Reconnaissance optique de caractères

![Un robot lisant un journal](./images/ocr.jpg)

Un défi courant en matière de vision par ordinateur consiste à détecter et à interpréter du texte dans une image. Ce type de traitement est souvent appelé *reconnaissance optique de caractères* (OCR).

## Utiliser le service Vision par ordinateur pour lire du texte dans une image

Le service cognitif **Vision par ordinateur** prend en charge les tâches d’OCR, notamment :

- Une API **OCR** que vous pouvez utiliser pour lire du texte dans plusieurs langues. Cette API peut être utilisée de manière synchrone et fonctionne bien lorsque vous devez détecter et lire une petite quantité de texte dans une image.
- Une API **Lecture** optimisée pour les documents plus volumineux. Cette API est utilisée de manière asynchrone et peut être utilisée pour les textes imprimés et manuscrits.

Vous pouvez utiliser ce service en créant soit une ressource **Vision par ordinateur** soit une ressource **Cognitive Services**.

Si vous ne l’avez pas encore fait, créez une ressource **Cognitive Services** dans votre abonnement Azure.

> **Remarque** : Si vous disposez déjà d’une ressource Cognitive Services, il suffit d’ouvrir sa page **Démarrage rapide** dans le portail Azure et de copier sa clé et son point de terminaison dans la cellule ci-dessous. Sinon, suivez les étapes ci-dessous pour en créer une.

1. Dans un autre onglet du navigateur, ouvrez le portail Azure à l’adresse https://portal.azure.com, en vous connectant avec votre compte Microsoft.

2. Cliquez sur le bouton **&#65291; Créer une ressource**, recherchez *Cognitive Services*, et créez une ressource **Cognitive Services** avec les paramètres suivants :
    - **Abonnement** : *Votre abonnement Azure*.
    - **Groupe de ressources** : *Sélectionnez ou créez un groupe de ressources portant un nom unique*.
    - **Région** : *Choisissez une région disponible* :
    - **Nom** : *Saisissez un nom unique*.
    - **Niveau tarifaire** : S0
    - **Je confirme avoir lu et compris les avis** : Sélectionné.
3. Attendez la fin du déploiement. Ensuite, accédez à votre ressource Cognitive Services et, sur la page **Aperçu**, cliquez sur le lien permettant de gérer les clés du service. Vous aurez besoin du point de terminaison et des clés pour vous connecter à votre ressource Cognitive Services à partir d’applications clientes.

### Obtenir la clé et le point de terminaison de votre ressource Cognitive Services

Pour utiliser votre ressource Cognitive Services, les applications clientes ont besoin de son point de terminaison et de sa clé d’authentification :

1. Dans le portail Azure, sur la page **Clés et Point de terminaison** de votre ressource Cognitive Services, copiez la **Clé 1** de votre ressource et collez-la dans le code ci-dessous, en remplaçant **YOUR_COG_KEY**.
2. Copiez le **point de terminaison** de votre ressource et collez-le dans le code ci-dessous, en remplaçant **YOUR_COG_ENDPOINT**.
3. Exécutez le code dans la cellule ci-dessous en cliquant sur le bouton **Exécuter la cellule** (&#9655;) (à gauche de la cellule).

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

Maintenant que vous avez configuré la clé et le point de terminaison, vous pouvez utiliser votre ressource de service de vision par ordinateur pour extraire du texte d’une image.

Commençons par l’API **OCR**, qui vous permet d’analyser de manière synchrone une image et de lire le texte qu’elle contient. Dans ce cas, vous disposez d’une image publicitaire pour l’entreprise de vente au détail fictive Northwind Traders qui contient du texte. Exécutez la cellule ci-dessous pour le lire... 

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import os
%matplotlib inline

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Read the image file
image_path = os.path.join('data', 'ocr', 'advert.jpg')
image_stream = open(image_path, "rb")

# Use the Computer Vision service to find text in the image
read_results = computervision_client.recognize_printed_text_in_stream(image_stream)

# Process the text line by line
for region in read_results.regions:
    for line in region.lines:

        # Read the words in the line of text
        line_text = ''
        for word in line.words:
            line_text += word.text + ' '
        print(line_text.rstrip())

# Open image to display it.
fig = plt.figure(figsize=(7, 7))
img = Image.open(image_path)
draw = ImageDraw.Draw(img)
plt.axis('off')
plt.imshow(img)

Le texte trouvé dans l’image est organisé en une structure hiérarchique de régions, de lignes et de mots, et le code les lit pour récupérer les résultats.

Dans les résultats, affichez le texte qui a été lu au-dessus de l’image. 

## Afficher les cases de délimitation

Les résultats incluent également les coordonnées de la *case de délimitation* pour les lignes de texte et les mots individuels trouvés dans l’image. Exécutez la cellule ci-dessous pour voir les cases de délimitation des lignes de texte dans l’image publicitaire que vous avez récupérée ci-dessus.

In [ ]:
# Open image to display it.
fig = plt.figure(figsize=(7, 7))
img = Image.open(image_path)
draw = ImageDraw.Draw(img)

# Process the text line by line
for region in read_results.regions:
    for line in region.lines:

        # Show the position of the line of text
        l,t,w,h = list(map(int, line.bounding_box.split(',')))
        draw.rectangle(((l,t), (l+w, t+h)), outline='magenta', width=5)

        # Read the words in the line of text
        line_text = ''
        for word in line.words:
            line_text += word.text + ' '
        print(line_text.rstrip())

# Show the image with the text locations highlighted
plt.axis('off')
plt.imshow(img)

Dans le résultat, la case de délimitation de chaque ligne de texte est représentée par un rectangle sur l’image.

## Utiliser l’API Lire

L’API OCR que vous avez utilisée précédemment fonctionne bien pour les images contenant une petite quantité de texte. Lorsque vous devez lire des textes plus volumineux, tels que des documents numérisés, vous pouvez utiliser l’API **Lire**. Ce processus nécessite plusieurs étapes :

1. Soumettez une image au service Vision par ordinateur pour qu’elle soit lue et analysée de manière asynchrone.
2. Attendez que l’opération d’analyse se termine.
3. Récupérez les résultats de l’analyse.

Exécutez la cellule suivante pour utiliser ce processus afin de lire le texte d’une lettre numérisée adressée au directeur d’un magasin Northwind Traders.

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image
import time
import os
%matplotlib inline

# Read the image file
image_path = os.path.join('data', 'ocr', 'letter.jpg')
image_stream = open(image_path, "rb")

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Submit a request to read printed text in the image and get the operation ID
read_operation = computervision_client.read_in_stream(image_stream,
                                                      raw=True)
operation_location = read_operation.headers["Operation-Location"]
operation_id = operation_location.split("/")[-1]

# Wait for the asynchronous operation to complete
while True:
    read_results = computervision_client.get_read_result(operation_id)
    if read_results.status not in [OperationStatusCodes.running]:
        break
    time.sleep(1)

# If the operation was successfuly, process the text line by line
if read_results.status == OperationStatusCodes.succeeded:
    for result in read_results.analyze_result.read_results:
        for line in result.lines:
            print(line.text)

# Open image and display it.
print('\n')
fig = plt.figure(figsize=(12,12))
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

Examinez les résultats. Il y a une transcription complète de la lettre, qui consiste principalement en un texte imprimé avec une signature manuscrite. L’image originale de la lettre est affichée sous les résultats de l’OCR (vous devrez peut-être faire défiler l’écran pour la voir).

## Lisez le texte manuscrit

Dans l’exemple précédent, la demande d’analyse de l’image spécifiait un mode de reconnaissance de texte qui optimisait l’opération pour le texte *imprimé*. Notez que malgré cela, la signature manuscrite a été lue.

Cette capacité à lire du texte manuscrit est extrêmement utile. Par exemple, supposons que vous ayez écrit une note contenant une liste de courses, et que vous souhaitiez utiliser une application sur votre téléphone pour lire la note et transcrire le texte qu’elle contient.

Exécutez la cellule ci-dessous pour voir un exemple d’une opération de lecture d’une liste de courses manuscrite.

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image
import time
import os
%matplotlib inline

# Read the image file
image_path = os.path.join('data', 'ocr', 'note.jpg')
image_stream = open(image_path, "rb")

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Submit a request to read printed text in the image and get the operation ID
read_operation = computervision_client.read_in_stream(image_stream,
                                                      raw=True)
operation_location = read_operation.headers["Operation-Location"]
operation_id = operation_location.split("/")[-1]

# Wait for the asynchronous operation to complete
while True:
    read_results = computervision_client.get_read_result(operation_id)
    if read_results.status not in [OperationStatusCodes.running]:
        break
    time.sleep(1)

# If the operation was successfuly, process the text line by line
if read_results.status == OperationStatusCodes.succeeded:
    for result in read_results.analyze_result.read_results:
        for line in result.lines:
            print(line.text)

# Open image and display it.
print('\n')
fig = plt.figure(figsize=(12,12))
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

## Informations complémentaires

Pour plus d’informations sur l’utilisation du service Vision par ordinateur pour l’OCR, consultez [la documentation Vision par ordinateur](https://docs.microsoft.com/fr-fr/azure/cognitive-services/computer-vision/concept-recognizing-text)